In [2]:
#Librerías
import gurobipy as gp
from gurobipy import GRB

#PARÁMETROS DEL MODELO

**n** = número de jobs

**m** = número de máquinas

**a_i,j,h** = indica si la operación O_j,h puede realizarse en la máquina "i" (1 = si, 0 = no)

**P_i,j,h**= tiempo de procesamiento si O_j, se ejecuta en la máquina "i"

**L** = un número muy grande, para restricciones con Big-M

#VARIABLES DEL MODELO

**y[i,j,o]**: si la operación "o" del trabajo "j" se asigna a la máquina "i"
    Es la desición de asginación de operacioens a máquinas.

**x[i,j,o,k]**: si la operacion "o" del trabajo "j" ocupa la posición "k" en la máquina "i"
    Permite modelar el orden de ejecución (secuencia) dentro de cada máquina.

**t[j,h]**: Tiempo de inicio del procesamiento de la operación O_j,h

**Tm_i,k**: El instante de tiempo en que la máquina "i" empieza a trabajar en su k-ésima operación.

**k_i**: Número de operaciones asignadas a la máquina "i".

**Ps[j,h]**: Tiempo de procesa,miento de la operación O_j,h luego de ser asignada a una máquina.

**Cmax**: Tiempo total de finalización del último trabajo.

*x e y son variables Binarias; t, T, Ps y Cmax son variables Continuas.*

#RESTRICCIONES (por definir):

**1:** El tiempo total del sistema debe ser mayor o igual al término de cada trabajo individual.

**2:** “selecciona el tiempo de procesamiento correcto” según la máquina elegida.

**3:** La operación h + 1 no puede empezar antes de que la operación h haya terminado

**4:** El inicio del slot siguiente debe ser después de que termine la operación que está en el slot actual. (Las máquinas tienen slots)

**5:** Hace coincidir el reloj de la máquina con el inicio de la operación asignada (Cada máquina tiene su propio “tiempo interno”, Tm_i,k)

**6:** El inicio del slot debe ser igual o después del inicio real de la operación que se ejecuta ahí.

** 5 y 6 juntas:** El modelo sincroniza los relojes de la máquina con los inicios de las operaciones que realmente se ejecutan ahí.

**7:** Impide asignar una operación a una máquina que no sea capaz de ejecutarla.

**8:** Cada posición (slot) en cada máquina tenga exactamente una operación asignada.

**9:** Exactamente una máquina por operación

**10:** Si una operación j,h se asigna a una máquina i, entonces debe ocupar exactamente un slot k de esa máquina. Si una operación no está asignada a la máquina i,
entonces no puede aparecer en ningún slot de esa máquina.

**11:** El tiempo de inicio de cada operación j,h no puede ser negativo.

**12:** No negatividad del tiempo de proceso efectivo

**13:** no negatividad del “reloj” de la máquina. Fue inicializada con lowerbound = 0, asi que la restricción es redundante (pero se mantendrá)

**14:** x_i,j,h,k es 0 o 1

**15:** y_i,j,h es 0 o 1

#Ejemplo de Instancia

5 6 ---> 5 = número de trabajos (J), 6 = número de máquinas (M).

3 3 0 147 1 123 2 145 2 3 140 1 130 2 3 150 4 160 ---> 3 = número de operaciones, H[j].

3 2 0 214 2 150 2 2 87 1 66 2 4 178 5 95

3 2 0 87 1 62 2 2 180 3 105 3 3 190 4 100 5 153

3 2 0 87 1 65 1 4 173 2 3 145 5 136

3 3 1 123 2 145 0 128 3 2 86 3 65 4 47 2 4 110 5 85


Son 5 filas, una por cada trabajo.
Ojo, luego de H[j], lo que vemos es "el número de máquinas para esa operación". Luego de eso, vienen pares {máquina, tiempo de procesamiento}

Visualización: H = operaciones, Máquina = M, Tiempo de procesamiento = p
3 2 0 87 1 62 2 2 180 3 105 3 3 190 4 100 5 153

H[3] num_maq_paraH[0] M0 p0 M1 p1 num_maq_paraH[1] M3 p3 M4 p4 num_maq_paraH[3] M5 p5 M6 p6 M7 p7



In [3]:
def read_fattahi_instance(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()

    # Primera línea: número de jobs y máquinas
    parts = lines[0].split()
    J_total = int(parts[0])
    M_total = int(parts[1])

    # Índices
    J = list(range(J_total))
    M = list(range(M_total))

    # Parámetros
    H = {}
    a = {}
    p = {}

    line_index = 1  # comenzamos después de la cabecera

    for j in J:
        tokens = list(map(int, lines[line_index].split()))
        line_index += 1

        pos = 0
        H[j] = tokens[pos]
        pos += 1

        for h in range(H[j]):
            num_machines = tokens[pos]
            pos += 1

            # Inicializamos a = 0 para TODAS las máquinas
            for i in M:
                a[(i,j,h)] = 0
                p[(i,j,h)] = 0

            # Leemos pares (máquina, tiempo)
            for _ in range(num_machines):
                machine = tokens[pos]
                time = tokens[pos+1]
                pos += 2

                a[(machine, j, h)] = 1
                p[(machine, j, h)] = time

    # Calcular K[i] = cantidad de operaciones que puede procesar
    K = {
        i: sum(a[(i,j,h)] for j in J for h in range(H[j]))
        for i in M
    }

    # BIGM = suma del tiempo máximo por operación
    BIGM = sum(
        max(p[(i,j,h)] for i in M if a[(i,j,h)] == 1)
        for j in J for h in range(H[j])
    )

    # Empaquetar todo
    data = {
        'J': J,
        'M': M,
        'H': H,
        'a': a,
        'p': p,
        'K': K,
        'BIGM': BIGM
    }

    return data


In [ ]:
from xml.parsers.expat import model


def solve_fjsp(data, timelimit=3600):

    #Parámetros:

    J = data['J']           # lista de trabajos
    M = data['M']           # lista de máquinas
    H = data['H']           # operaciones por trabajo (dict)
    a = data['a']       # máquinas capaces
    p = data['p']           # tiempos p[i,j,h]
    K = {i: sum(a[(i, j, h)] for j in J for h in range(H[j])) for i in M} #la cota superior es la cantidad de máquinas que pueden realizar el trabajo Ojh
    BIGM = sum(max(p[(i,j,h)] for i in M if a[(i,j,h)] == 1) for j in J for h in range(H[j]))



    #Modelo:

    model = gp.Model("FJSP_Fattahi_Tarea_3") #instancia del modelo
    model.Params.TimeLimit = timelimit #define límite de tiempo



    #Variables:
    y = model.addVars([(i,j,h) for i in M for j in J for h in range(H[j])],
                  vtype=GRB.BINARY, name="y")
    x = model.addVars([(i,j,h,k) for i in M for j in J for h in range(H[j]) for k in range(K[i])],
                      vtype=GRB.BINARY, name="x")
    t = model.addVars([(j,h) for j in J for h in range(H[j])], lb=0.0, name="t")
    Tm = model.addVars([(i,k) for i in M for k in range(K[i])], lb=0.0, name="Tm")
    Ps = model.addVars([(j,h) for j in J for h in range(H[j])], lb=0.0, name="Ps")
    Cmax = model.addVar(lb=0.0, name="Cmax")



    #Función Objetivo:
    model.setObjective(Cmax, GRB.MINIMIZE)




    #Restricciones:

    #(1): Cmax mayor o igual que el final de cada trabajo
    for j in J:
        last = H[j] - 1
        model.addConstr(Cmax >= t[j,last] + Ps[j,last], name=f"cmax_{j}")




    for j in J:
        for h in range(H[j]):
            #(2):
            model.addConstr(
                Ps[j,h] == gp.quicksum(p[(i,j,h)] * y[i,j,h] for i in M),
                name=f"ptime_{j}_{h}"
                )
            #(9):
            model.addConstr(
                gp.quicksum(y[i,j,h] for i in M) == 1,
                name=f"one_machine_per_operation_{j}_{h}"
            )
            #(11):
            model.addConstr(
                t[j,h] >= 0,
                name=f"start_time_nonnegative_{j}_{h}"
                )
            #(12):
            model.addConstr(
                Ps[j,h] >= 0,
                name=f"processing_time_nonnegative_{j}_{h}"
            )

        for h in range(H[j]-1):
            #(3):
            model.addConstr(t[j,h] + Ps[j,h] <= t[j,h+1], name=f"prec_{j}_{h}")










    for i in M:
        for j in J:
            for h in range(H[j]):
                #(7):
                model.addConstr(
                y[i,j,h] <= a[(i,j,h)],
                name=f"machine_capability_{i}_{j}_{h}"
                )
                #(10):
                model.addConstr(
                    gp.quicksum(x[i,j,h,k] for k in range(K[i])) == y[i,j,h],
                    name=f"link_y_x_{i}_{j}_{h}"
                )
                #Parece que las restricciones (15) no son necesarias al definir y como variable binaria,
                #pero estarían acá.


                for k in range(K[i] - 1):
                    #(4)
#IMPORATANTE: el paper en vez de p, utiliza Ps, pero eso lo vuelve un problema NO lineal, por lo que se utiliza p directamente.
#Se ha comprobado que los resultados son los mismos teoricamente hablando. Hay diferencias de flotantes, 
# por ejemplo en el sfjs03 tenemos que con Ps da exactamente 221, y con p da 220.9999999777954... detalles.
                    model.addConstr(
                        Tm[i,k] + p[i,j,h] * x[i,j,h,k] <= Tm[i, k + 1],
                        name=f"slot_link_{i}_{j}_{h}_{k}"
                    )
                for k in range (K[i]):
                    #(5)
                    model.addConstr(
                        Tm[i,k] <= t[j,h] + (1 - x[i,j,h,k]) * BIGM,
                        name=f"start_time_link_{i}_{j}_{h}_{k}"
                    )
                    #(6)
                    model.addConstr(
                        Tm[i,k] + (1 - x[i,j,h,k]) * BIGM >= t[j,h],
                        name=f"start_time_link2_{i}_{j}_{h}_{k}"
                    )

                    #Parece que las restricciones (14) no son necesarias al definir x como variable binaria,
                    #pero estarían acá.


    for i in M:
        for k in range(K[i]):
            #(8): gpt dice que debe ser <= 1, pero el paper dice que es = 1
            model.addConstr(
                gp.quicksum(x[i,j,h,k] for j in J for h in range(H[j])) <= 1,
                name=f"one_operation_per_slot_{i}_{k}"

            )

            #(13):
            model.addConstr(
                Tm[i,k] >= 0,
                name=f"machine_time_nonnegative_{i}_{k}"
            )







        # ==========================
    # Optimizar
    # ==========================
    model.optimize()

    # ==========================
    # Mostrar resultados
    # ==========================
    if model.Status in (GRB.OPTIMAL, GRB.TIME_LIMIT):
        print(f"\nCmax = {Cmax.X}")

        # Mostrar máquina de cada operación
        print("\nAsignación de máquinas:")
        for j in J:
            for h in range(H[j]):
                for i in M:
                    if y[i,j,h].X > 0.5:
                        print(f"Trabajo {j}, Operación {h}: máquina {i}")

        # Mostrar start times
        print("\nTiempos de inicio:")
        for j in J:
            for h in range(H[j]):
                print(f"t[{j},{h}] = {t[j,h].X}")
    if model.Status == GRB.OPTIMAL:
        print("Estado: Óptimo")
        print("Cmax =", Cmax.X)
    elif model.Status == GRB.TIME_LIMIT:
        print("Estado: Se alcanzó el tiempo límite")
        print("Cmax =", Cmax.X)
    else:
        print("Estado:", model.Status)

    result = {
        "jobs": len(J),
        "machines": len(M),
        "variables": model.NumVars,
        "constraints": model.NumConstrs,
        "status": model.Status,
        "runtime": model.Runtime,
        "objective_value": Cmax.X if model.SolCount > 0 else None,
    }
    return model, result

In [7]:
#ahora lee desde el json

import json

with open("instances.json", "r") as f:
    instances = json.load(f)

def get_instance_info(name, instances):
    for inst in instances:
        if inst["name"] == name:
            return inst
    raise ValueError(f"Instancia {name} no encontrada")

In [ ]:
# Elegir una instancia
name = "sfjs01"
info = get_instance_info(name, instances)
txt_path = "instances/" + info["path"]

data = read_fattahi_instance(txt_path)

# correremos de una a una las instancias
model, result = solve_fjsp(data)
print("Resultado instancia:", name)
print(result)



Set parameter TimeLimit to value 3600
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12450HX, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  3600

Optimize a model with 140 rows, 57 columns and 392 nonzeros
Model fingerprint: 0xa1b421d6
Variable types: 17 continuous, 40 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 33 rows and 8 columns
Presolve time: 0.00s
Presolved: 107 rows, 49 columns, 347 nonzeros
Variable types: 13 continuous, 36 integer (36 binary)
Found heuristic solution: objective 126.0000000

Root relaxation: objective 6.600000e+01, 39 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 E

In [ ]:
# Calcular GAP
Csolver = model.getVarByName("Cmax").X

if info.get("optimum") is not None:  #si el json tiene el valor del "óptimo", se compara con él
    OPT = info["optimum"]
    gap = (Csolver - OPT) / OPT * 100
    print(f"OPTIMO = {OPT}")
    print(f"GAP respecto al OPTIMO: {gap:.2f}%")

elif "bounds" in info:
    LB = info["bounds"]["lower"]
    UB = info["bounds"]["upper"]

    gap_LB = (Csolver - LB) / Csolver * 100
    gap_UB = (UB - Csolver) / UB * 100

    print(f"Lower bound = {LB}")
    print(f"Upper bound = {UB}")
    print(f"GAP respecto al LOWER BOUND: {gap_LB:.2f}%")  #si no, se comparan estos gap
    print(f"GAP respecto al UPPER BOUND: {gap_UB:.2f}%")

else:
    print("La instancia no tiene OPT ni bounds en el JSON.")
#en resumen está el GAPopt, GAPlb y GAPub. Hay que mencionar eso en la tarea.

OPTIMO = 66
GAP respecto al OPTIMO: 0.00%
